In [1]:
#importing libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import math

In [2]:
#load the NSL_KDD dataset
train=pd.read_csv('dataset/train_TCP1.csv')
test=pd.read_csv('dataset/Test_TCP1.csv')

#size of train and test data
print(train.shape,test.shape)

# concatenate traina and test data
dataset=pd.concat([test,train])

#size of whole dataset
print(dataset.shape)

#print(dataset.head(2))
#print(dataset.columns)
X = dataset.iloc[:, :-1].values
print(X)
y = dataset.iloc[:, 41:42].values
print(y)

(95788, 42) (12865, 42)
(108653, 42)
[[0 'tcp' 'private' ... 0.0 1.0 1.0]
 [0 'tcp' 'private' ... 0.0 1.0 1.0]
 [2 'tcp' 'ftp_data' ... 0.0 0.0 0.0]
 ...
 [0 'tcp' 'smtp' ... 0.0 0.01 0.0]
 [0 'tcp' 'klogin' ... 1.0 0.0 0.0]
 [0 'tcp' 'ftp_data' ... 0.0 0.0 0.0]]
[['DDoS']
 ['DDoS']
 ['normal']
 ...
 ['normal']
 ['DDoS']
 ['normal']]


In [3]:
#Encoding of dataset
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1, 2, 3])], remainder = 'passthrough')
X= ct.fit_transform(X)
print(X)

[[1.0 0.0 0.0 ... 0.0 1.0 1.0]
 [1.0 0.0 0.0 ... 0.0 1.0 1.0]
 [1.0 0.0 0.0 ... 0.0 0.0 0.0]
 ...
 [1.0 0.0 0.0 ... 0.0 0.01 0.0]
 [1.0 0.0 0.0 ... 1.0 0.0 0.0]
 [1.0 0.0 0.0 ... 0.0 0.0 0.0]]


In [4]:
from sklearn.preprocessing import LabelEncoder
le_y = LabelEncoder()
y[:, 0] = le_y.fit_transform(y[:, 0])
y= y.astype(int)
y = y.ravel()
print(y)

[0 0 1 ... 1 0 1]


In [5]:
# min_max normalisation between (a,b)
from sklearn.preprocessing import minmax_scale
X = minmax_scale(X, feature_range=(0,1), axis=0)
print(X)

[[0.   0.   0.   ... 0.   1.   1.  ]
 [0.   0.   0.   ... 0.   1.   1.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.01 0.  ]
 [0.   0.   0.   ... 1.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]]


In [6]:
# function to calculate shannon entropy
def cal_entropy(column):
   counts=np.bincount(column)
   probabilites=counts / len(column)
   entropy=0
   for prob in probabilites:
      if(prob)>0:
        entropy+=prob*math.log(prob,2)
   return -entropy 

In [7]:
# function to calculte MAD adaptive threshold technique
def thres_MAD(data):
  a=3
  from numpy import mean, absolute
  MAD =mean(absolute(data - mean(data)))
  thres= mean(data)+a*MAD
  return thres

In [8]:
print(X.shape,y.shape)

(108653, 106) (108653,)


In [9]:
# form a list of attribute that has to deleted further
list=[]
for j in range(0,106):
   col=np.array(X[:,j])
   col=col.astype(int)
   if  cal_entropy(col)<thres_MAD(col):
           list.append(j)


print(list)

[17, 20, 41, 45, 58, 62, 66, 76, 89, 90, 91, 92, 93, 95, 96, 97, 98, 102, 103, 104, 105]


In [10]:
# delete the attribute that have less entropy than adaptive threshold value(THR)
X = np.delete(X, list, axis=1)

In [11]:
print(X.shape)

(108653, 85)


In [12]:
# splitting of dataset in train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)
print(X_train)
print(X_test)
print(y_train)
print(y_test)


[[0.   0.   0.   ... 0.07 0.   0.  ]
 [0.   0.   0.   ... 0.08 0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.08 0.   0.  ]
 [0.   0.   0.   ... 0.   0.14 0.07]
 [0.   0.   0.   ... 0.   0.01 0.03]]
[[0.   0.   0.   ... 0.   0.02 0.05]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.07 0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.17 0.06 0.27]]
[0 0 1 ... 0 1 1]
[1 1 1 ... 0 0 1]


In [13]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [14]:
# base classifier RandomForest
y_train = y_train.ravel()
clf1 = RandomForestClassifier(n_estimators=100)
clf1.fit(X_train, y_train)
y_pred1 = clf1.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred1)
cm.print_stats()
#Root mean square error between actual and predicted class
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred1))
print("RMSE= ",RMSE)

population: 32596
P: 18302
N: 14294
PositiveTest: 18301
NegativeTest: 14295
TP: 18288
TN: 14281
FP: 13
FN: 14
TPR: 0.9992350562780024
TNR: 0.9990905274940535
PPV: 0.9992896563029343
NPV: 0.999020636586219
FPR: 0.000909472505946551
FDR: 0.0007103436970657341
FNR: 0.0007649437219975958
ACC: 0.9991716775064425
F1_score: 0.9992623555446275
MCC: 0.9983179383013289
informedness: 0.9983255837720559
markedness: 0.9983102928891534
prevalence: 0.5614799361884893
LRP: 1098.697376495213
LRN: 0.0007656400505730435
DOR: 1435005.098901099
FOR: 0.0009793634137810422
RMSE=  0.02878059230727352


In [15]:
# base classifier GradientBoosting
clf2 = GradientBoostingClassifier(n_estimators=100)
clf2.fit(X_train, y_train)
y_pred2 = clf2.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred2)
cm.print_stats()
#Root mean square error between actual and predicted class
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred2))
print("RMSE= ",RMSE)

population: 32596
P: 18302
N: 14294
PositiveTest: 18344
NegativeTest: 14252
TP: 18286
TN: 14236
FP: 58
FN: 16
TPR: 0.9991257786034313
TNR: 0.9959423534350077
PPV: 0.9968382032272133
NPV: 0.9988773505472917
FPR: 0.0040576465649923045
FDR: 0.0031617967727867423
FNR: 0.000874221396568681
ACC: 0.997729782795435
F1_score: 0.9979806800196475
MCC: 0.9953917902695483
informedness: 0.9950681320384391
markedness: 0.995715553774505
prevalence: 0.5614799361884893
LRP: 246.2328255061629
LRN: 0.0008777831302720376
DOR: 280516.6982758621
FOR: 0.001122649452708392
RMSE=  0.047646796372526216


In [16]:
# SVM
from sklearn.svm import SVC  
clf3= SVC(kernel='linear') 
clf3.fit(X_train, y_train) 
y_pred3 = clf3.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred3)
cm.print_stats()
#Root mean square error between actual and predicted class
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred3))
print("RMSE= ",RMSE)

population: 32596
P: 18302
N: 14294
PositiveTest: 18869
NegativeTest: 13727
TP: 18160
TN: 13585
FP: 709
FN: 142
TPR: 0.992241285105453
TNR: 0.9503987687141457
PPV: 0.9624251417669193
NPV: 0.9896554236176878
FPR: 0.049601231285854204
FDR: 0.03757485823308072
FNR: 0.007758714894547044
ACC: 0.9738925021475028
F1_score: 0.9771058082914099
MCC: 0.9473485501095887
informedness: 0.9426400538195987
markedness: 0.9520805653846072
prevalence: 0.5614799361884893
LRP: 20.00436802439682
LRN: 0.008163641568101248
DOR: 2450.422137905004
FOR: 0.010344576382312232
RMSE=  0.1615781478186244


In [17]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
clf4 = KNeighborsClassifier(n_neighbors = 17)
clf4.fit(X_train, y_train) 
y_pred4 = clf4.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred4)
cm.print_stats()
#Root mean square error between actual and predicted class
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred4))
print("RMSE= ",RMSE)

population: 32596
P: 18302
N: 14294
PositiveTest: 18361
NegativeTest: 14235
TP: 18222
TN: 14155
FP: 139
FN: 80
TPR: 0.9956288930171566
TNR: 0.9902756401287254
PPV: 0.9924296062305975
NPV: 0.9943800491745697
FPR: 0.00972435987127466
FDR: 0.007570393769402538
FNR: 0.004371106982843405
ACC: 0.9932813842189225
F1_score: 0.9940266753948122
MCC: 0.9863569904532943
informedness: 0.985904533145882
markedness: 0.9868096554051671
prevalence: 0.5614799361884893
LRP: 102.38503163156285
LRN: 0.00441403060492855
DOR: 23195.360611510794
FOR: 0.005619950825430277
RMSE=  0.08196716282193396


In [18]:
# meta classifier LogisticRegression
lr = LogisticRegression(solver='lbfgs')


In [19]:
# stack esemble method 
from mlxtend.classifier import StackingClassifier
sclf = StackingClassifier(classifiers=[clf1, clf2,clf4], 
                          meta_classifier=lr)

In [20]:
# train and test the model
sclf.fit(X_train, y_train)
y_pred = sclf.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred)
cm.print_stats()
#Root mean square error between actual and predicted class
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred))
print("RMSE= ",RMSE)


population: 32596
P: 18302
N: 14294
PositiveTest: 18299
NegativeTest: 14297
TP: 18288
TN: 14283
FP: 11
FN: 14
TPR: 0.9992350562780024
TNR: 0.9992304463411221
PPV: 0.9993988742554238
NPV: 0.9990207735888648
FPR: 0.0007695536588778508
FDR: 0.0006011257445762063
FNR: 0.0007649437219975958
ACC: 0.9992330347281875
F1_score: 0.999316958553045
MCC: 0.998442574968464
informedness: 0.9984655026191245
markedness: 0.9984196478442886
prevalence: 0.5614799361884893
LRP: 1298.4605358579788
LRN: 0.0007655328405960678
DOR: 1696152.6233766235
FOR: 0.0009792264111352033
RMSE=  0.027694137860068732
